# Import

In [1]:
%install-swiftpm-flags -c release
%install '.package(url: "https://github.com/JacopoMangiavacchi/SwiftCoreMLTools.git", from: "0.0.3")' SwiftCoreMLTools

Installing packages:
	.package(url: "https://github.com/JacopoMangiavacchi/SwiftCoreMLTools.git", from: "0.0.3")
		SwiftCoreMLTools
With SwiftPM flags: ['-c', 'release']
Working in: /tmp/tmpf6rj8rvh/swift-install
Fetching https://github.com/JacopoMangiavacchi/SwiftCoreMLTools.git
Fetching https://github.com/apple/swift-protobuf.git
Cloning https://github.com/JacopoMangiavacchi/SwiftCoreMLTools.git
Resolving https://github.com/JacopoMangiavacchi/SwiftCoreMLTools.git at 0.0.3
Cloning https://github.com/apple/swift-protobuf.git
Resolving https://github.com/apple/swift-protobuf.git at 1.8.0
[1/2] Compiling SwiftProtobuf AnyMessageStorage.swift
[2/3] Compiling SwiftCoreMLTools Activations.swift
[3/4] Compiling jupyterInstalledPackages jupyterInstalledPackages.swift
[4/4] Linking libjupyterInstalledPackages.so
Initializing Swift...
Installation complete!


In [2]:
import Foundation
import TensorFlow
import SwiftCoreMLTools

# Data Ingestion

In [3]:
let trainCSV = try String(contentsOfFile:"../data/train.csv", encoding: String.Encoding.utf8)
let testCSV = try String(contentsOfFile:"../data/test.csv", encoding: String.Encoding.utf8)

let trainRecords: [[Float]] = trainCSV.split(separator: "\n").map{ String($0).split(separator: ",").compactMap{ Float(String($0)) } }
let testRecords: [[Float]] = testCSV.split(separator: "\n").map{ String($0).split(separator: ",").compactMap{ Float(String($0)) } }

let numTrainRecords = trainRecords.count
let numTrainColumns = trainRecords[0].count
let numTestRecords = testRecords.count
let numTestColumns = testRecords[0].count

print(numTrainRecords, numTrainColumns, numTestRecords, numTestColumns)

let xTrain = trainRecords.map{ Array($0[0..<numTrainColumns-1]) }
let yTrain = trainRecords.map{ [$0[numTrainColumns-1]] }
let xTest = testRecords.map{ Array($0[0..<numTestColumns-1]) }
let yTest = testRecords.map{ [$0[numTestColumns-1]] }

print(xTrain.count, xTrain[0].count, yTrain.count, yTrain[0].count,
      xTest.count, xTest[0].count, yTest.count, yTest[0].count)

let xAllTrain = Array(xTrain.joined())
let yAllTrain = Array(yTrain.joined())
let xAllTest = Array(xTest.joined())
let yAllTest = Array(yTest.joined())

print(xAllTrain.count, yAllTrain.count, xAllTest.count, yAllTest.count)

405 14 101 14
405 13 405 1 101 13 101 1
5265 405 1313 101


In [4]:
let XTrain = Tensor<Float>(xAllTrain).reshaped(to: TensorShape([numTrainRecords, numTrainColumns-1]))
let YTrain = Tensor<Float>(yAllTrain).reshaped(to: TensorShape([numTrainRecords, 1]))
let XTest = Tensor<Float>(xAllTest).reshaped(to: TensorShape([numTestRecords, numTestColumns-1]))
let YTest = Tensor<Float>(yAllTest).reshaped(to: TensorShape([numTestRecords, 1]))

print(XTrain.shape, YTrain.shape, XTest.shape, YTest.shape)

[405, 13] [405, 1] [101, 13] [101, 1]


# Model

In [5]:
struct RegressionModel: Layer {
    var layer1 = Dense<Float>(inputSize: 13, outputSize: 64, activation: relu)
    var layer2 = Dense<Float>(inputSize: 64, outputSize: 32, activation: relu)
    var layer3 = Dense<Float>(inputSize: 32, outputSize: 1)
    
    @differentiable
    func callAsFunction(_ input: Tensor<Float>) -> Tensor<Float> {
        return input.sequenced(through: layer1, layer2, layer3)
    }
}

var model = RegressionModel()

# Training

In [6]:
let optimizer = RMSProp(for: model, learningRate: 0.001)
Context.local.learningPhase = .training

In [7]:
let epochCount = 500
let batchSize = 32
let numberOfBatch = Int(ceil(Double(xTrain.count) / Double(batchSize)))
let shuffle = true

func mae(predictions: Tensor<Float>, truths: Tensor<Float>) -> Float {
    return abs(Tensor<Float>(predictions - truths)).mean().scalarized()
}

In [8]:
for epoch in 1...epochCount {
    var epochLoss: Float = 0
    var epochMAE: Float = 0
    var batchCount: Int = 0
    var batchArray = Array(repeating: false, count: numberOfBatch)
    for batch in 0..<numberOfBatch {
        var r = batch
        if shuffle {
            while true {
                r = Int.random(in: 0..<numberOfBatch)
                if !batchArray[r] {
                    batchArray[r] = true
                    break
                }
            }
        }
        
        let batchStart = r * batchSize
        let batchEnd = min(xTrain.count, batchStart + batchSize)
        let (loss, grad) = model.valueWithGradient { (model: RegressionModel) -> Tensor<Float> in
            let logits = model(XTrain[batchStart..<batchEnd])
            return meanSquaredError(predicted: logits, expected: YTrain[batchStart..<batchEnd])
        }
        optimizer.update(&model, along: grad)
        
        let logits = model(XTrain[batchStart..<batchEnd])
        epochMAE += mae(predictions: logits, truths: YTrain[batchStart..<batchEnd])
        epochLoss += loss.scalarized()
        batchCount += 1
    }
    epochMAE /= Float(batchCount)
    epochLoss /= Float(batchCount)

    print("Epoch \(epoch): MSE: \(epochLoss), MAE: \(epochMAE)")
}

Epoch 1: MSE: 564.0721, MAE: 21.880272
Epoch 2: MSE: 509.87088, MAE: 20.597134
Epoch 3: MSE: 450.3012, MAE: 19.11409
Epoch 4: MSE: 382.68845, MAE: 17.311522
Epoch 5: MSE: 310.86276, MAE: 15.244508
Epoch 6: MSE: 239.10733, MAE: 12.9609375
Epoch 7: MSE: 172.21379, MAE: 10.560276
Epoch 8: MSE: 118.50304, MAE: 8.408708
Epoch 9: MSE: 81.78389, MAE: 6.7140603
Epoch 10: MSE: 59.660343, MAE: 5.5374274
Epoch 11: MSE: 46.48149, MAE: 4.768462
Epoch 12: MSE: 38.120743, MAE: 4.223739
Epoch 13: MSE: 32.454426, MAE: 3.8931606
Epoch 14: MSE: 28.738583, MAE: 3.6474686
Epoch 15: MSE: 26.42635, MAE: 3.4820552
Epoch 16: MSE: 24.593666, MAE: 3.3668518
Epoch 17: MSE: 23.16613, MAE: 3.2889097
Epoch 18: MSE: 21.928768, MAE: 3.1672094
Epoch 19: MSE: 21.037416, MAE: 3.0863268
Epoch 20: MSE: 20.00581, MAE: 3.037299
Epoch 21: MSE: 19.294664, MAE: 2.9592059
Epoch 22: MSE: 18.48949, MAE: 2.9172251
Epoch 23: MSE: 17.788433, MAE: 2.8383403
Epoch 24: MSE: 17.177328, MAE: 2.7904055
Epoch 25: MSE: 16.63548, MAE: 2.75453

Epoch 202: MSE: 4.5702715, MAE: 1.4203385
Epoch 203: MSE: 4.463578, MAE: 1.3926375
Epoch 204: MSE: 4.5659256, MAE: 1.404947
Epoch 205: MSE: 4.47126, MAE: 1.3938258
Epoch 206: MSE: 4.430917, MAE: 1.3921229
Epoch 207: MSE: 4.48835, MAE: 1.3818285
Epoch 208: MSE: 4.363862, MAE: 1.3845539
Epoch 209: MSE: 4.3752813, MAE: 1.3671514
Epoch 210: MSE: 4.325841, MAE: 1.3769853
Epoch 211: MSE: 4.317844, MAE: 1.3728204
Epoch 212: MSE: 4.2878966, MAE: 1.3695439
Epoch 213: MSE: 4.292905, MAE: 1.3655349
Epoch 214: MSE: 4.293278, MAE: 1.3591272
Epoch 215: MSE: 4.2996664, MAE: 1.3564249
Epoch 216: MSE: 4.1255646, MAE: 1.3590674
Epoch 217: MSE: 4.218949, MAE: 1.345712
Epoch 218: MSE: 4.226391, MAE: 1.3460295
Epoch 219: MSE: 4.197092, MAE: 1.3405137
Epoch 220: MSE: 4.202805, MAE: 1.3353404
Epoch 221: MSE: 4.090663, MAE: 1.3408328
Epoch 222: MSE: 4.1967344, MAE: 1.3233522
Epoch 223: MSE: 4.0542965, MAE: 1.3244395
Epoch 224: MSE: 4.1697235, MAE: 1.3335387
Epoch 225: MSE: 4.162044, MAE: 1.3182836
Epoch 226: 

Epoch 401: MSE: 2.2191577, MAE: 0.8772091
Epoch 402: MSE: 2.0851808, MAE: 0.87745243
Epoch 403: MSE: 2.207153, MAE: 0.87336606
Epoch 404: MSE: 2.1359825, MAE: 0.8695854
Epoch 405: MSE: 2.159112, MAE: 0.8846785
Epoch 406: MSE: 2.1176775, MAE: 0.8701508
Epoch 407: MSE: 2.1649356, MAE: 0.8614153
Epoch 408: MSE: 2.1034129, MAE: 0.8568605
Epoch 409: MSE: 2.1044614, MAE: 0.87246174
Epoch 410: MSE: 2.1276555, MAE: 0.8586957
Epoch 411: MSE: 2.035919, MAE: 0.8613918
Epoch 412: MSE: 2.0582001, MAE: 0.8574234
Epoch 413: MSE: 2.1237154, MAE: 0.865313
Epoch 414: MSE: 2.1331217, MAE: 0.85997325
Epoch 415: MSE: 2.045267, MAE: 0.86661047
Epoch 416: MSE: 2.0859008, MAE: 0.85422254
Epoch 417: MSE: 2.0223234, MAE: 0.8444697
Epoch 418: MSE: 2.1031237, MAE: 0.8484201
Epoch 419: MSE: 2.0958006, MAE: 0.8465413
Epoch 420: MSE: 2.07511, MAE: 0.84495074
Epoch 421: MSE: 2.0311158, MAE: 0.847485
Epoch 422: MSE: 2.031372, MAE: 0.84110737
Epoch 423: MSE: 2.099362, MAE: 0.85182434
Epoch 424: MSE: 1.9495481, MAE: 0.8

# Export

In [9]:
let coremlModel = Model(version: 4,
                        shortDescription: "Regression",
                        author: "Jacopo Mangiavacchi",
                        license: "MIT",
                        userDefined: ["SwiftCoremltoolsVersion" : "0.0.3"]) {
    Input(name: "input", shape: [13], featureType: .Double)
    Output(name: "output", shape: [1], featureType: .Double)
    NeuralNetwork {
        InnerProduct(name: "dense1",
                     input: ["input"],
                     output: ["outDense1"],
                     weights: model.layer1.weight.flattened().scalars,
                     bias: model.layer1.bias.flattened().scalars,
                     inputChannels: 13,
                     outputChannels: 64,
                     updatable: false)
        ReLu(name: "Relu1",
             input: ["outDense1"],
             output: ["outRelu1"])
        InnerProduct(name: "dense2",
                     input: ["outRelu1"],
                     output: ["outDense2"],
                     weights: model.layer2.weight.flattened().scalars,
                     bias: model.layer2.bias.flattened().scalars,
                     inputChannels: 64,
                     outputChannels: 32,
                     updatable: false)
        ReLu(name: "Relu2",
             input: ["outDense2"],
             output: ["outRelu2"])
        InnerProduct(name: "dense3",
                     input: ["outRelu2"],
                     output: ["output"],
                     weights: model.layer3.weight.flattened().scalars,
                     bias: model.layer3.bias.flattened().scalars,
                     inputChannels: 32,
                     outputChannels: 1,
                     updatable: false)
    }
}